# Example of dynamic network creation

This notebook shows how to create a simple network first, using the standard techniques that we have already met, and then using a technique for dynamic class and object creation, binding classes and objects to dictionaries to enable iterative creatio of the class definitions and the object instantiations.

In [ ]:
from owlready2 import  *

onto = get_ontology("http://www.dummy.info/new.owl")

In [ ]:
with onto:
   

    class e1(Thing): pass
    class e2(Thing): pass
    class e3(Thing): pass

    class r1(ObjectProperty):
        name = 'relation1'
        domain = [e1,e2]
        range = [e2,e3]

    class r2(ObjectProperty):
        name = 'relation2'
        domain = [e2,e3]
        range = [e1]

    n1 = e2(name='node1')
    n2 = e3(name='node2')
    n3 = e1(name='node3')

    n1.r1 = [n1,n2]
    n2.r2 = [n3]
    n3.r2 = [n2]

    print("Standard way")
    print(f"{n1.name} is linked to {[k.name for k in n1.r1]} by {r1.__name__}")
    print(f"{n2.name} is linked to {[k.name for k in n2.r2]} by {r2.name}")
    print(f"{n3.name} is linked to {[k.name for k in n3.r2]} by {r2.name}")

    ## Now we explore a different approach based on dynamic creation of classes. This approach is useful when we are creating a large network from an external source of data.

    # Define the entity types
    Entities = {
        'ent1': types.new_class('Entity1', (Thing,)),
        'ent2': types.new_class('Entity2', (Thing,)),
        'ent3': types.new_class('Entity3', (Thing,)),
    }

    # Relations, defined here to show how to do it but we will not use them
    Relations = dict()
    Relations['r1'] = types.new_class('Relation1', (ObjectProperty,))
    Relations['r1'].name = 'rel1'
    Relations['r1'].domain = [Entities['ent1'], Entities['ent2']]
    Relations['r1'].domain = [Entities['ent2'], Entities['ent3']]
    
    Relations['r2'] = types.new_class('Relation2', (ObjectProperty,))
    Relations['r1'].name = 'rel2'
    Relations['r2'].domain = [Entities['ent2'], Entities['ent3']]
    Relations['r2'].domain = [Entities['ent1']]

    #define the nodes in the graph
    Nodes = {
        'no1': Entities['ent2'](name='dnode1'),
        'no2': Entities['ent3'](name='dnode2'),
        'no3': Entities['ent1'](name='dnode3')
    }



    # Define the relation. We do not, here use the relations as defined above for technical reasons but instead bind to a new object property 
    Nodes['no1'].rel = [Nodes['no2']]
    Nodes['no1'].rel.append(Nodes['no1'])
    Nodes['no2'].rel = [Nodes['no3']]
    Nodes['no3'].rel = [Nodes['no2']]
    
    print("\nDynamic way")
    print(f"{Nodes['no1'].name} is linked to {[k.name for k in Nodes['no1'].rel]}")
    print(f"{Nodes['no2'].name} is linked to {[k.name for k in Nodes['no2'].rel]}")
    print(f"{Nodes['no3'].name} is linked to {[k.name for k in Nodes['no3'].rel]}")


onto.save('links.rdf')